In [1]:
!nvidia-smi

Wed Dec  3 05:21:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import random, numpy as np, torch

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    # Optional but good enough for a project
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Seed set to {seed}")

set_seed(42)


Seed set to 42


In [3]:
import os, json
from pathlib import Path
from getpass import getpass

def setup_kaggle_with_token():
    kaggle_dir = Path.home() / ".kaggle"
    config_path = kaggle_dir / "kaggle.json"

    if config_path.exists():
        print(f"Kaggle credentials already set at: {config_path}")
        return

    KAGGLE_USERNAME = os.environ.get("KAGGLE_USERNAME") or "jeremywijaya"
    KAGGLE_KEY = getpass("Paste your Kaggle API token (KGAT_...): ").strip()

    os.environ["KAGGLE_USERNAME"] = KAGGLE_USERNAME
    os.environ["KAGGLE_KEY"] = KAGGLE_KEY

    kaggle_dir.mkdir(exist_ok=True)
    with open(config_path, "w") as f:
        json.dump({"username": KAGGLE_USERNAME, "key": KAGGLE_KEY}, f)
    try:
        os.chmod(config_path, 0o600)
    except PermissionError:
        pass

    print(f"Kaggle credentials set. Config at: {config_path}")

setup_kaggle_with_token()


Kaggle credentials set. Config at: /root/.kaggle/kaggle.json


In [ ]:
!pip install -q kaggle

from pathlib import Path

RAW_ROOT = Path("./datasets/cardd_raw")
YOLO_ROOT = Path("./datasets/cardd")

RAW_ROOT.mkdir(parents=True, exist_ok=True)

if not (RAW_ROOT / "train.json").exists():
    print("Downloading CarDD dataset...")
    !kaggle datasets download -d issamjebnouni/cardd -p ./datasets/cardd_raw --unzip
    print("✅ Download complete")
else:
    print("CarDD raw dataset already present, skipping download.")


Dataset URL: https://www.kaggle.com/datasets/issamjebnouni/cardd
License(s): unknown
 99% 2.79G/2.81G [00:30<00:00, 29.4MB/s]
100% 2.81G/2.81G [00:30<00:00, 97.7MB/s]
✅ Download complete


In [5]:
import shutil

RAW_ROOT = Path("./datasets/cardd_raw")
YOLO_ROOT = Path("./datasets/cardd")

YOLO_ROOT.mkdir(parents=True, exist_ok=True)

for split in ["train", "val", "test"]:
    src = RAW_ROOT / split
    dst = YOLO_ROOT / split
    if not src.exists():
        print(f"{split}: source {src} not found, skipping move.")
        continue
    if dst.exists():
        shutil.rmtree(dst)
    shutil.move(str(src), str(dst))
    print(f"Moved {split} -> {dst}")

print("Final YOLO root:", YOLO_ROOT.resolve())


Moved train -> datasets/cardd/train
Moved val -> datasets/cardd/val
Moved test -> datasets/cardd/test
Final YOLO root: /content/datasets/cardd


In [6]:
print(os.listdir("./datasets"))
# ['val.json', 'test.json', 'image_info.xlsx', 'train', 'test', 'train.json', 'val']


['cardd_raw', 'cardd']


In [7]:
import json
from collections import defaultdict

def build_cat_mapping(json_path: Path):
    with open(json_path, "r") as f:
        data = json.load(f)
    cats = sorted(data["categories"], key=lambda c: c["id"])
    # Map COCO category_id -> 0..(n-1)
    cat_to_class = {c["id"]: i for i, c in enumerate(cats)}
    print("Category mapping (id -> class idx):", cat_to_class)
    return cat_to_class

CAT_TO_CLASS = build_cat_mapping(RAW_ROOT / "train.json")

def convert_split(json_path: Path, split_dir: Path):
    print(f"\nConverting {json_path.name} -> {split_dir}")
    with open(json_path, "r") as f:
        data = json.load(f)

    images = {img["id"]: img for img in data["images"]}
    anns_by_img = defaultdict(list)
    for ann in data["annotations"]:
        anns_by_img[ann["image_id"]].append(ann)

    img_dir = split_dir / "images"
    lbl_dir = split_dir / "labels"
    img_dir.mkdir(exist_ok=True)
    lbl_dir.mkdir(exist_ok=True)

    for img_file in split_dir.glob("*.jpg"):
        img_file.rename(img_dir / img_file.name)

    for img_id, img_info in images.items():
        file_name = img_info["file_name"]
        w, h = img_info["width"], img_info["height"]

        label_lines = []
        for ann in anns_by_img[img_id]:
            x, y, bw, bh = ann["bbox"]
            x_c = (x + bw / 2) / w
            y_c = (y + bh / 2) / h
            bw_n = bw / w
            bh_n = bh / h

            cls = CAT_TO_CLASS[ann["category_id"]]
            label_lines.append(f"{cls} {x_c:.6f} {y_c:.6f} {bw_n:.6f} {bh_n:.6f}")

        stem = Path(file_name).stem
        with open(lbl_dir / f"{stem}.txt", "w") as f:
            f.write("\n".join(label_lines))

    print("  Done.")

convert_split(RAW_ROOT / "train.json", YOLO_ROOT / "train")
convert_split(RAW_ROOT / "val.json",   YOLO_ROOT / "val")
convert_split(RAW_ROOT / "test.json",  YOLO_ROOT / "test")


Category mapping (id -> class idx): {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5}

Converting train.json -> datasets/cardd/train
  Done.

Converting val.json -> datasets/cardd/val
  Done.

Converting test.json -> datasets/cardd/test
  Done.


In [8]:
print("train images:", len(list((YOLO_ROOT / "train/images").glob("*.jpg"))))
print("train labels:", len(list((YOLO_ROOT / "train/labels").glob("*.txt"))))

for split in ["train", "val", "test"]:
    imgs = len(list((YOLO_ROOT / f"{split}/images").glob("*.jpg")))
    lbls = len(list((YOLO_ROOT / f"{split}/labels").glob("*.txt")))
    print(f"{split}: {imgs} images, {lbls} labels")
    assert imgs == lbls, f"{split}: {imgs} images vs {lbls} labels (mismatch!)"


train images: 2816
train labels: 2816
train: 2816 images, 2816 labels
val: 810 images, 810 labels
test: 374 images, 374 labels


In [9]:
import yaml

yaml_config = {
    "path": "../datasets/cardd",   # from notebook location
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "names": {
        0: "dent",
        1: "scratch",
        2: "crack",
        3: "glass_shatter",
        4: "lamp_broken",
        5: "tire_flat",
    },
}

with open("cardd.yaml", "w") as f:
    yaml.dump(yaml_config, f, default_flow_style=False, sort_keys=False)

print(open("cardd.yaml").read())


path: ../datasets/cardd
train: train/images
val: val/images
test: test/images
names:
  0: dent
  1: scratch
  2: crack
  3: glass_shatter
  4: lamp_broken
  5: tire_flat



In [10]:
print("YOLO root:", Path("./datasets/cardd").resolve())
print("train/images sample:", len(list((Path("./datasets/cardd")/"train/images").glob("*.jpg"))))
print("train/labels sample:", len(list((Path("./datasets/cardd")/"train/labels").glob("*.txt"))))


YOLO root: /content/datasets/cardd
train/images sample: 2816
train/labels sample: 2816


# Baseline

In [ ]:
!pip install -q ultralytics pyyaml

In [ ]:
from ultralytics import YOLO
import torch

DEVICE = 0 if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

model = YOLO("yolo11n.pt")
model.train(
    data="cardd.yaml",
    epochs=20,          # start smaller; later you can do 50–100 on GPU
    imgsz=640,
    batch=8,            # safer for low RAM / CPU
    seed=42,
    project="runs/cardd",
    name="yolo11n_baseline",
    device=DEVICE,
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Using device: 0
Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=cardd.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b895290af30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [12]:
metrics = model.val(data="cardd.yaml", split="test")
print("mAP50:", metrics.box.map50, "mAP50-95:", metrics.box.map)


Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,583,322 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 79.4±33.1 MB/s, size: 731.5 KB)
val: Scanning /content/datasets/cardd/test/labels... 374 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 374/374 397.8it/s 0.9s0.0s
val: New cache created: /content/datasets/cardd/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 24/24 3.1it/s 7.7s0.3s
                   all        374        785      0.675      0.641      0.673      0.522
                  dent        157        236      0.602      0.538      0.584      0.326
               scratch        183        307      0.545      0.508      0.546        0.3
                 crack         48         70       0.43      0.229      0.229      0.124
         glass_shatter         71         71      0.928      0.

# Experiment 1 - Same model

In [ ]:
model = YOLO("yolo11n.pt")
results = model.train(
    data="cardd.yaml",
    epochs=100,
    imgsz=640,
    batch=8,
    seed=42,
    project="runs/cardd",
    name="yolo11n_100e",
    device=DEVICE,
    save_period=10,
)


In [ ]:
metrics = model.val(data="cardd.yaml", split="test")
print(metrics.box.map50, metrics.box.map)


# Experiment 2 (YOLO11s)

In [ ]:
model = YOLO("yolo11s.pt")
results = model.train(
    data="cardd.yaml",
    epochs=100,
    imgsz=640,
    batch=8,
    seed=42,
    project="runs/cardd",
    name="yolo11s_100e",
    device=DEVICE,
    save_period=10,
)


In [ ]:
metrics = model.val(data="cardd.yaml", split="test")
print(metrics.box.map50, metrics.box.map)
